In [1]:
from numpy import *
import time 

In [21]:
x = random.random((10000,10000))
v = random.random((10000,1))

%timeit foo = dot(x, v)


10 loops, best of 3: 28 ms per loop


In [23]:
# test sparse matrix multiplication
from scipy import sparse
from scipy.sparse import lil_matrix
x = sparse.rand(10000,10000)
x=x.tobsr()

%timeit foo = x.dot(v)


1000 loops, best of 3: 855 µs per loop


In [24]:
def SpMV_viaMKL( A, x ):
     '''
     Wrapper to Intel's SpMV
     (Sparse Matrix-Vector multiply)
     For medium-sized matrices, this is 4x faster
     than scipy's default implementation
     Stephen Becker, April 24 2014
     stephen.beckr@gmail.com
     '''

     import numpy as np
     import scipy.sparse as sparse
     from ctypes import POINTER,c_void_p,c_int,c_char,c_double,byref,cdll
     mkl = cdll.LoadLibrary("libmkl_rt.so")

     SpMV = mkl.mkl_cspblas_dcsrgemv
     # Dissecting the "cspblas_dcsrgemv" name:
     # "c" - for "c-blas" like interface (as opposed to fortran)
     #    Also means expects sparse arrays to use 0-based indexing, which python does
     # "sp"  for sparse
     # "d"   for double-precision
     # "csr" for compressed row format
     # "ge"  for "general", e.g., the matrix has no special structure such as symmetry
     # "mv"  for "matrix-vector" multiply

     m, n = A.shape

     # The data of the matrix
     data    = A.data.ctypes.data_as(POINTER(c_double))
     indptr  = A.indptr.ctypes.data_as(POINTER(c_int))
     indices = A.indices.ctypes.data_as(POINTER(c_int))

     # Allocate output, using same conventions as input
     nVectors = 1
     if x.ndim is 1:
        y = np.empty(m,dtype=np.double,order='F')
     elif x.shape[1] is 1:
        y = np.empty((m,1),dtype=np.double,order='F')
        

   

     np_x = x.ctypes.data_as(POINTER(c_double))
     np_y = y.ctypes.data_as(POINTER(c_double))
     # now call MKL. This returns the answer in np_y, which links to y
     SpMV(byref(c_char("N")), byref(c_int(m)),data ,indptr, indices, np_x, np_y ) 

     return y

In [27]:
x  = x.tocsr()

%timeit foo = SpMV_viaMKL(A=x,x=v)


The slowest run took 21.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 413 µs per loop


In [11]:
sparse.isspmatrix_csr(x.tocsr())

True